# COLLABORATIV FILTERING (implicit Fake_users)

Test du model pour la recommendation sur les faux utilisateurs

## Imports

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.sparse import coo_matrix, csr_matrix

from sklearn.metrics import roc_auc_score

import itertools

from tqdm import tqdm

## Data import

In [2]:
df_train = pd.read_csv('filtered data/filtered_rating_fm_dataset_train.csv', index_col=0)

In [3]:
files = ['last_fm_fake_user(1001)_jazz.csv',
         'last_fm_fake_user(1002)_classic.csv',
         'last_fm_fake_user(1003)_pop.csv',
         'last_fm_fake_user(1004)_rock.csv',
         'last_fm_fake_user(1005)_rap.csv']

for file in files:
    df_temp = pd.read_csv(f'filtered data/fake_user/{file}', index_col=0)
    df_temp['rating']=100 # a revoir
    df_train = pd.concat([df_train, df_temp])

## Data preparation

In [4]:
df_track = df_train.groupby(['track_id']).first().reset_index()

In [5]:
df_gb = df_train.groupby(['user_id','track_id']).mean().reset_index()

In [74]:
item = np.array(df_gb[['track_id']].values).T[0]
user = np.array(df_gb[['user_id']].values).T[0]
weight = np.array(df_gb[['rating']].values).T[0]

In [76]:
mat_music = csr_matrix((weight,(item, user))).T
mat_music.shape

(1006, 80600)

## Modèle

In [77]:
import implicit

# Initialisation du modèle
model_music = implicit.als.AlternatingLeastSquares(factors=40,
                                                   use_native=True, 
                                                   use_cg=True, 
                                                   calculate_training_loss=True, 
                                                   num_threads=1, 
                                                   iterations=30)

# Entrainemt du model item/user/confidence weights
model_music.fit(mat_music)

F:\anaconda3\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.016002416610717773 seconds
  warnings.warn(


  0%|          | 0/30 [00:00<?, ?it/s]

In [88]:
def reco_user_music(user_id, mat, n=10, filter=False):
    if filter:
        f= list(df_train[df_train['user_id']==user_id]['track_id'].unique())
    else:
        f = list()
        
    track_ids = model_music.recommend(user_id, mat.tocsr()[user_id], n, filter_already_liked_items=filter)
    
    for id,score in zip(track_ids[0], track_ids[1]):
        a = df_track[df_track['track_id']==id]
        track = a['track_name'].values[0]
        artist = a['artist_name'].values[0]
        print(f'{str(score)[:4]}--> {artist}: {track}')

### Utilisateur Jazz 1001

In [89]:
reco_user_music(1001, mat_music, filter=False)

0.88--> Nina Simone: Don'T Let Me Be Misunderstood
0.88--> Nina Simone: Feeling Good
0.86--> Frank Sinatra: The Best Is Yet To Come
0.86--> Miles Davis: Blue In Green
0.85--> Norah Jones: Come Away With Me
0.84--> Ray Charles: Hit The Road Jack
0.83--> Norah Jones: Lonestar
0.83--> Norah Jones: Wish I Could
0.83--> Frank Sinatra: I Get A Kick Out Of You
0.82--> Duke Ellington: Sophisticated Lady


In [90]:
reco_user_music(1001, mat_music, filter=True)

0.70--> Frank Sinatra: My Way
0.69--> John Coltrane: Blue Train
0.69--> Miles Davis: Move
0.67--> Wes Montgomery: 4 On 6
0.66--> John Coltrane: Giant Steps
0.65--> Norah Jones: Turn Me On
0.64--> Frank Sinatra: That'S Life
0.63--> Frank Sinatra: Strangers In The Night
0.62--> Art Blakey & The Jazz Messengers: Blues March
0.62--> Miles Davis: Summertime


### Utilisateur Classique 1002

In [91]:
reco_user_music(1002, mat_music, filter=False)

0.82--> Ludwig Van Beethoven: Für Elise
0.74--> Wolfgang Amadeus Mozart: Requiem
0.72--> Wolfgang Amadeus Mozart: Recordare
0.71--> Ludwig Van Beethoven: 5Th Symphony
0.71--> Wolfgang Amadeus Mozart: Dies Irae
0.70--> Wolfgang Amadeus Mozart: Tuba Mirum
0.70--> Johann Sebastian Bach: Air
0.70--> Robert Schumann: Trumerei
0.69--> Antonio Vivaldi: Allegro
0.69--> Antonio Vivaldi: Concerto In G


In [92]:
reco_user_music(1002, mat_music, filter=True)

0.48--> Yann Tiersen: La Dispute
0.48--> Joy Division: Insight
0.47--> Yann Tiersen: Sur Le Fil
0.46--> Samuel Barber: Adagio For Strings
0.46--> Yann Tiersen: La Valse Des Vieux Os
0.46--> Yann Tiersen: Soir De Fête
0.46--> Yann Tiersen: Comptine D'Un Autre Été : L'Après-Midi
0.45--> Yann Tiersen: J'Y Suis Jamais Allé
0.45--> Yann Tiersen: Comptine D'Un Autre Été: L'Après Midi
0.45--> Yann Tiersen: La Valse Des Monstres


### Utilisateur Pop 1003

In [14]:
reco_user_music(1003, mat_music, filter=False)

1.00--> Madonna: Hung Up
0.98--> Madonna: Sorry
0.98--> Madonna: Jump
0.96--> Rihanna: Umbrella (Feat. Jay-Z)
0.94--> Miley Cyrus: See You Again
0.93--> Justin Timberlake: Losing My Way
0.93--> Britney Spears: Toxic
0.92--> Justin Timberlake: Sexyback
0.92--> Madonna: Miles Away
0.92--> Gwen Stefani: Early Winter


In [15]:
reco_user_music(1003, mat_music, filter=True)

0.98--> Madonna: Sorry
0.98--> Madonna: Jump
0.92--> Madonna: Miles Away
0.92--> Britney Spears: Gimme More
0.90--> The Pussycat Dolls: Buttons
0.90--> Madonna: Give It 2 Me
0.86--> Britney Spears: Circus
0.85--> Christina Aguilera: Ain'T No Other Man
0.84--> Gwen Stefani: Wind It Up
0.83--> Christina Aguilera: Keeps Gettin' Better


### Utilisateur Rock 1004

In [16]:
reco_user_music(1004, mat_music, filter=False)

0.94--> Korn: Freak On A Leash
0.92--> Linkin Park: Bleed It Out
0.91--> Disturbed: Inside The Fire
0.90--> Slipknot: Psychosocial
0.90--> Korn: A.D.I.D.A.S.
0.90--> Linkin Park: Numb
0.89--> Linkin Park: Lying From You
0.89--> Rage Against The Machine: How I Could Just Kill A Man
0.89--> Nirvana: Come As You Are
0.88--> Linkin Park: What I'Ve Done


In [17]:
reco_user_music(1004, mat_music, filter=True)

0.83--> Nirvana: Heart-Shaped Box
0.79--> Korn: Falling Away From Me
0.79--> Coldplay: Violet Hill
0.76--> Beastie Boys: Sabotage
0.76--> Limp Bizkit: Hold On
0.75--> Limp Bizkit: Break Stuff
0.75--> Limp Bizkit: Hot Dog
0.75--> Slipknot: Duality
0.74--> Slipknot: Pulse Of The Maggots
0.72--> Rage Against The Machine: Bulls On Parade


### Utilisateur Rap 1005

In [18]:
reco_user_music(1005, mat_music, filter=False)

0.92--> Kanye West: Paranoid (Feat. Mr. Hudson)
0.92--> Kanye West: Welcome To Heartbreak (Feat. Kid Cudi)
0.90--> Kanye West: Good Morning
0.89--> Kanye West: Can'T Tell Me Nothing
0.89--> Kanye West: Touch The Sky
0.88--> Kanye West: Family Business
0.87--> Lupe Fiasco: Superstar
0.87--> Eminem: Kim
0.86--> 2Pac: Me Against The World
0.86--> Kanye West: Gold Digger


In [19]:
reco_user_music(1005, mat_music, filter=True)

0.82--> Kanye West: Street Lights
0.81--> Kanye West: I Wonder
0.81--> Kanye West: Good Life (Feat. T-Pain)
0.80--> Kanye West: Robocop
0.80--> Kanye West: Coldest Winter
0.79--> Kanye West: Stronger
0.79--> Kanye West: Amazing (Feat. Young Jeezy)
0.78--> Kanye West: Big Brother
0.78--> Kanye West: Everything I Am
0.74--> Kanye West: Pinocchio Story (Freestyle Live From Singapore)
